In [1]:
import openai
import pandas as pd
import time

In [ ]:
# OpenAI API 키를 설정합니다.
openai.api_key = ''

In [3]:
#excel 파일명 list
excel_fname = []
excel_fname = ['company_ingredient_to_name.xlsx',
               'company_product_to_equivalent.xlsx',
               'name_name_to_prohibition.xlsx',
               'name_to_age.xlsx',
               'name_to_efficacy.xlsx',
               'name_to_general_info.xlsx',
               'name_to_ingredient_en.xlsx',
               'name_to_ingredient_kr.xlsx',
               'name_to_pregnancy.xlsx',
               'name_to_usage.xlsx']

In [4]:
'''
api_call : 
    요약        :   gpt api에 질문을 하는 함수
    특이사항    :   Timeout이나 server overload 문제 발생시 에러처리해서 다음 질문에 대한 호출 진행.
                    만약 에러 발생시 retry 횟수만큼 다시 질문 시도. 질문 시도 횟수는 retry_cnt로 확인
    prompt      :   You are a knowledgeable pharmacist. 
                    Answer the questions about medications accurately. 
                    Do not use phrases like ' consult with a doctor or pharmacist.' 
                    Please respond in Korean.
    매개변수    :   
                    question : 질문
                    num      : 몇번째 질문인지 확인하기위한 카운터
                    retry    : 오류 발생시 재시도 횟수
    반환값      :
                    문자열과 실수(답변과 걸린 시간)
                    정상적으로 response 받은 경우에는 답변과 걸린 시간을 반환
                    비정상 종료시 "error" 문자열과 걸린 시간 반환
'''


def api_call(question,num,retry):
    retry_cnt = 0
    while retry_cnt < retry:
        try:
            start_time = time.time()  # API 호출 전 시간 기록
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",  # 모델을 지정
                messages=[              # 질문을 입력
                    {"role": "system", "content": "You are a knowledgeable pharmacist. Answer the questions about medications accurately. Do not use phrases like ' consult with a doctor or pharmacist.' Please answer in English."},
                    {"role": "user", "content": question}
                ],  
                max_tokens=300,  # 최대 토큰 수
                                # 해당 토큰을 만나면 중지,
                request_timeout = timeout
            )
            end_time = time.time()  # API 호출 후 시간 기록
            time_taken = end_time - start_time  # 걸린 시간 계산
            print("Q",num,": "+ question + " answered : ",time_taken)
            return response.choices[0].message['content'], time_taken
        #에외처리
        except Exception as e:
            end_time = time.time()  # API 호출 후 시간 기록
            time_taken = end_time - start_time  # 걸린 시간 계산
            print(f"An error occured on attempt {retry_cnt + 1} : {e}", time_taken)
            retry_cnt = retry_cnt + 1
            time.sleep(1)    #오류 발생시 1초 대기후 다시 질문
    return "Failed to get an answer", time_taken

In [5]:
#parameters
#오류 발생시 재시도 횟수
retry = 5
#타임아웃 시간 설정
timeout = 40

In [6]:
#파일마다 api를 호출해서 답변을 받는다.
for fname in excel_fname:
    #엑셀 파일 pandas dataframe 변환
    df = pd.read_excel(fname)
    #question list에 질문 저장
    questions = df['crude_question']
    answers = []
    time_taken_list = []
    num = 0
    total_time = 0.0
    #질문에 대한 답변을 받아서 answers list에 저장.
    for question in questions:
        answer, time_taken = api_call(question,num,retry)
        answers.append(answer)
        total_time = total_time + time_taken
        time_taken_list.append(time_taken)
        num = num+1
    #pandas dataframe에 gpt 답변 추가
    df['gpt-3.5-turbo answer'] = answers
    df['time_taken'] = time_taken_list
    df['total_time'] = total_time
    #dataframe 엑셀 변환
    df.to_excel(fname, index = False)

Q 0 : 동아에스티의 엔테카비르 약 뭐가 있어? answered :  22.299047708511353
Q 1 : 한국코러스의 테라조신염산염 약 뭐가 있어? answered :  18.824074745178223
Q 2 : 명인제약의 레보설피리드 약 뭐가 있어? answered :  5.262819051742554
Q 3 : 제이더블유중외제약의 미녹시딜 약 뭐가 있어? answered :  16.847190141677856
Q 4 : 한올바이오파마의 로수바스타틴칼슘 약 뭐가 있어? answered :  12.246115684509277
Q 5 : 환인제약의 메틸페니데이트염산염 약 뭐가 있어? answered :  19.343554735183716
Q 6 : 오스코리아제약의 애엽95%에탄올 연조엑스(20→1) 약 뭐가 있어? answered :  21.335892915725708
Q 7 : 서흥의 아세트아미노펜 약 뭐가 있어? answered :  24.3660306930542
Q 8 : 한국신약의 고삼가루 약 뭐가 있어? answered :  36.38335633277893
Q 9 : 원광제약의 진피 약 뭐가 있어? answered :  13.808761835098267
Q 10 : 바이넥스의 이토프리드염산염 약 뭐가 있어? answered :  25.060899257659912
Q 11 : 보령바이오파마의 발사르탄 약 뭐가 있어? answered :  22.39225459098816
Q 12 : 코오롱제약의 디클로페낙나트륨 약 뭐가 있어? answered :  15.882782220840454
Q 13 : 한국유씨비제약의 히드록시진염산염 약 뭐가 있어? answered :  31.791121006011963
Q 14 : 유케이케미팜의 플로목세프나트륨 약 뭐가 있어? answered :  21.355828523635864
Q 15 : 환인제약의 이토프리드염산염 약 뭐가 있어? answered :  16.27512288093567
Q 16 : 구주제약의 시타글

Q 120 : 제이더블유중외제약의 아세트아미노펜 약 뭐가 있어? answered :  35.279921531677246
Q 121 : 대한뉴팜의 시프로플록사신 약 뭐가 있어? answered :  11.984638690948486
Q 122 : 옵투스제약의 아르기닌티디아시케이트 약 뭐가 있어? answered :  15.904121160507202
Q 123 : 아이엔테라퓨틱스의 펙수프라잔염산염 약 뭐가 있어? answered :  10.102458477020264
Q 124 : 휴온스의 아미노필린수화물 약 뭐가 있어? answered :  14.557144403457642
Q 125 : 에이치디엑스의 플루오르화나트륨 F-18액 약 뭐가 있어? answered :  24.486738681793213
Q 126 : 한국오가논의 알렌드론산나트륨 약 뭐가 있어? answered :  23.26395297050476
Q 127 : 테라젠이텍스의 시타글립틴인산염수화물 약 뭐가 있어? answered :  23.81159496307373
Q 128 : 동아에스티의 글리메피리드 약 뭐가 있어? answered :  23.571415185928345
Q 129 : 동구바이오제약의 실로스타졸 약 뭐가 있어? answered :  11.987077474594116
Q 130 : 동아에스티의 가바펜틴 약 뭐가 있어? answered :  21.750665426254272
Q 131 : 대한뉴팜의 바클로펜 약 뭐가 있어? answered :  23.82915759086609
An error occured on attempt 1 : Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=40) 40.0056266784668
Q 132 : 삼성제약의 시타글립틴인산염수화물 약 뭐가 있어? answered :  37.96271753311157
Q 133 : 마더

Q 37 : 한국릴리(유)의 탈츠프리필드시린지주과/와 동일한 약 뭐야? answered :  35.65106201171875
Q 38 : 대한약품공업의 대한글루콘산칼슘주_과/와 동일한 약 뭐야? answered :  31.979901552200317
Q 39 : 녹십자의 아이비글로불린에스엔주[말토즈 첨가 사람 면역글로불린과/와 동일한 약 뭐야? answered :  23.704995155334473
Q 40 : 녹십자엠에스의 에취디졸비씨에이액_과/와 동일한 약 뭐야? answered :  8.595968246459961
Q 41 : 에이프로젠바이오로직스의 에이프로젠무피로신연고_과/와 동일한 약 뭐야? answered :  19.19252610206604
Q 42 : 한화제약의 안티비오캡슐과/와 동일한 약 뭐야? answered :  20.056225299835205
Q 43 : 제뉴파마의 히로손로션과/와 동일한 약 뭐야? answered :  21.26245641708374


KeyboardInterrupt: 